In [3]:
import torch
import evaluate
from tqdm import tqdm
import pandas as pd


Distane_Function = "MMD"
Variation = "normalized"

exact_match = {"dataset_index": [], "base": [], "lora": [], "pred": [], "error": []}
missing_loras = []

rouge = evaluate.load("rouge")

rouge_metric_results = {
    "dataset_index": [],
    "base": [],
    "lora": [],
    "pred": [],
    "error": [],
}

for index in tqdm(range(502)):
    try:
        pred = torch.load(
            f"R:\\Research\\FineTuning_LoRAs\\results\\{Distane_Function}\\{Variation}\\predicted_loras_outputs\\{index}.pt",
            weights_only=False,
        )

        lora = torch.load(
            f"R:\\Research\\FineTuning_LoRAs\\data\\Outputs\\base_lora\\{index}.pt",
            weights_only=False,
        )

        base = torch.load(
            f"R:\\Research\\FineTuning_LoRAs\\data\\Outputs\\base_model\\{index}.pt",
            weights_only=False,
        )

        labels = [item[0]["output"][0] for item in base]
        base_out = [
            item[1]["generated_text"][0][len(item[0]["input"]) :] for item in base
        ]
        lora_out = [
            item[1]["generated_text"][0][len(item[0]["input"]) :] for item in lora
        ]
        pred_out = [
            item[1]["generated_text"][0][len(item[0]["input"]) :] for item in pred
        ]

        rouge_metric_results["dataset_index"].append(index)
        rouge_metric_results["base"].append(
            rouge.compute(predictions=base_out, references=labels)
        )
        rouge_metric_results["lora"].append(
            rouge.compute(predictions=lora_out, references=labels)
        )
        rouge_metric_results["pred"].append(
            rouge.compute(predictions=pred_out, references=labels)
        )
        rouge_metric_results["error"].append("Good")

        pred_correct = []
        base_correct = []
        lora_correct = []
        for data_index in range(len(pred)):
            if (
                base[data_index][0]["output"][0].lower()
                in base[data_index][1]["generated_text"][0][-10:].lower()
            ):
                base_correct.append(1)
            else:
                base_correct.append(0)

            if (
                lora[data_index][0]["output"][0].lower()
                in lora[data_index][1]["generated_text"][0][-10:].lower()
            ):
                lora_correct.append(1)
            else:
                lora_correct.append(0)

            if (
                pred[data_index][0]["output"][0].lower()
                in pred[data_index][1]["generated_text"][0][-10:].lower()
            ):
                pred_correct.append(1)
            else:
                pred_correct.append(0)

        accuracy_base = sum(base_correct) / len(base_correct)
        accuracy_lora = sum(lora_correct) / len(lora_correct)
        accuracy_pred = sum(pred_correct) / len(pred_correct)
        print(f"Accuracy Table for model {index}:")
        print("Base Model Accuracy: ", accuracy_base)
        print("LoRA Model Accuracy: ", accuracy_lora)
        print("Predicted Model Accuracy: ", accuracy_pred)

        exact_match["dataset_index"].append(index)
        exact_match["base"].append(accuracy_base)
        exact_match["lora"].append(accuracy_lora)
        exact_match["pred"].append(accuracy_pred)
        exact_match["error"].append("Good")

    except Exception as e:
        rouge_metric_results["dataset_index"].append(index)
        rouge_metric_results["base"].append(0)
        rouge_metric_results["lora"].append(0)
        rouge_metric_results["pred"].append(0)
        rouge_metric_results["error"].append(e)

        exact_match["dataset_index"].append(index)
        exact_match["base"].append(0)
        exact_match["lora"].append(0)
        exact_match["pred"].append(0)
        exact_match["error"].append(e)
        missing_loras.append({"index": index, "error": e})

rouge_l = {"dataset": [], "base": [], "lora": [], "pred": []}
rouge_l_sum = {"dataset": [], "base": [], "lora": [], "pred": []}

for index in range(502):
    if rouge_metric_results["error"][index] == "Good":
        rouge_l["dataset"].append(index)
        rouge_l["base"].append(rouge_metric_results["base"][index]["rougeL"])
        rouge_l["lora"].append(rouge_metric_results["lora"][index]["rougeL"])
        rouge_l["pred"].append(rouge_metric_results["pred"][index]["rougeL"])

        rouge_l_sum["dataset"].append(index)
        rouge_l_sum["base"].append(rouge_metric_results["base"][index]["rougeLsum"])
        rouge_l_sum["lora"].append(rouge_metric_results["lora"][index]["rougeLsum"])
        rouge_l_sum["pred"].append(rouge_metric_results["pred"][index]["rougeLsum"])


results_name = Variation + "_" + Distane_Function
torch.save(rouge_metric_results, f"results/{results_name}_AllRouges.pt")

df_exact_match = pd.DataFrame.from_dict(exact_match)
df_exact_match["pred_is_better_than_base"] = (
    df_exact_match["pred"] > df_exact_match["base"]
)
df_exact_match["pred_is_better_than_lora"] = (
    df_exact_match["pred"] > df_exact_match["lora"]
)
df_exact_match["pred_diff_base"] = df_exact_match["pred"] - df_exact_match["base"]
df_exact_match["pred_diff_lora"] = df_exact_match["pred"] - df_exact_match["lora"]
df_exact_match.to_csv(f"{results_name}_ExactMatach.csv")

df_rouge_l = pd.DataFrame.from_dict(rouge_l)
df_rouge_l.to_csv(f"{results_name}_RougeL.csv")

df_rouge_l_sum = pd.DataFrame.from_dict(rouge_l_sum)
df_rouge_l_sum.to_csv(f"{results_name}_RougeLSum.csv")

  0%|          | 1/502 [00:00<01:36,  5.20it/s]

Accuracy Table for model 0:
Base Model Accuracy:  0.3076923076923077
LoRA Model Accuracy:  1.0
Predicted Model Accuracy:  0.7692307692307693


  0%|          | 2/502 [00:00<03:03,  2.72it/s]

Accuracy Table for model 1:
Base Model Accuracy:  0.42923076923076925
LoRA Model Accuracy:  0.7953846153846154
Predicted Model Accuracy:  0.42615384615384616


  1%|          | 3/502 [00:04<15:58,  1.92s/it]

...

Accuracy Table for model 498:
Base Model Accuracy:  0.07884615384615384
LoRA Model Accuracy:  0.9884615384615385
Predicted Model Accuracy:  0.06538461538461539
Accuracy Table for model 499:
Base Model Accuracy:  0.06666666666666667
LoRA Model Accuracy:  0.13333333333333333
Predicted Model Accuracy:  0.06666666666666667


100%|█████████▉| 501/502 [05:12<00:00,  2.65it/s]

Accuracy Table for model 500:
Base Model Accuracy:  0.0
LoRA Model Accuracy:  0.0
Predicted Model Accuracy:  0.0


100%|██████████| 502/502 [05:12<00:00,  1.60it/s]

Accuracy Table for model 501:
Base Model Accuracy:  0.3
LoRA Model Accuracy:  1.0
Predicted Model Accuracy:  0.7333333333333333


In [8]:
index = 24
pred = torch.load(
        f"R:\\Research\\FineTuning_LoRAs\\results\\{Distane_Function}\\nonlearnable\\predicted_loras_outputs\\{index}.pt",
        weights_only=False,
    )

lora = torch.load(
    f"R:\\Research\\FineTuning_LoRAs\\data\\Outputs\\base_lora\\{index}.pt",
    weights_only=False,
)

base = torch.load(
    f"R:\\Research\\FineTuning_LoRAs\\data\\Outputs\\base_model\\{index}.pt",
    weights_only=False,
)

In [9]:
base[1]

[{'input': 'Definition: In this task the focus is on physical knowledge about the world. Given the provided goal task in the input, describe a process that would lead to the asked outcome. This process often involves physical motions with objects, such as moving them, arranging them in a certain way, mixing them, shaking them, etc.\n\nPositive Example 1 -\nInput: How do you remove keys from a computer keyboard for cleaning?\nOutput: Place the tip of a knife under the corner of a key and gently twist to pop it off.\n\nPositive Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: by arranging paper towels on the counter and evenly spacing out each key to dry, face side up, on top of them.\n\nNegative Example 1 -\nInput: How do you remove keys?\nOutput: pry them off with a knife.\n\nNegative Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: Place them in the oven on low\n\nNow complete the following example -\nInput: Clean

In [13]:
base[5]

[{'input': 'Definition: In this task the focus is on physical knowledge about the world. Given the provided goal task in the input, describe a process that would lead to the asked outcome. This process often involves physical motions with objects, such as moving them, arranging them in a certain way, mixing them, shaking them, etc.\n\nPositive Example 1 -\nInput: How do you remove keys from a computer keyboard for cleaning?\nOutput: Place the tip of a knife under the corner of a key and gently twist to pop it off.\n\nPositive Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: by arranging paper towels on the counter and evenly spacing out each key to dry, face side up, on top of them.\n\nNegative Example 1 -\nInput: How do you remove keys?\nOutput: pry them off with a knife.\n\nNegative Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: Place them in the oven on low\n\nNow complete the following example -\nInput: To de

In [14]:
lora[5]

[{'input': 'Definition: In this task the focus is on physical knowledge about the world. Given the provided goal task in the input, describe a process that would lead to the asked outcome. This process often involves physical motions with objects, such as moving them, arranging them in a certain way, mixing them, shaking them, etc.\n\nPositive Example 1 -\nInput: How do you remove keys from a computer keyboard for cleaning?\nOutput: Place the tip of a knife under the corner of a key and gently twist to pop it off.\n\nPositive Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: by arranging paper towels on the counter and evenly spacing out each key to dry, face side up, on top of them.\n\nNegative Example 1 -\nInput: How do you remove keys?\nOutput: pry them off with a knife.\n\nNegative Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: Place them in the oven on low\n\nNow complete the following example -\nInput: To de

In [15]:
pred[5]

[{'input': 'Definition: In this task the focus is on physical knowledge about the world. Given the provided goal task in the input, describe a process that would lead to the asked outcome. This process often involves physical motions with objects, such as moving them, arranging them in a certain way, mixing them, shaking them, etc.\n\nPositive Example 1 -\nInput: How do you remove keys from a computer keyboard for cleaning?\nOutput: Place the tip of a knife under the corner of a key and gently twist to pop it off.\n\nPositive Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: by arranging paper towels on the counter and evenly spacing out each key to dry, face side up, on top of them.\n\nNegative Example 1 -\nInput: How do you remove keys?\nOutput: pry them off with a knife.\n\nNegative Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: Place them in the oven on low\n\nNow complete the following example -\nInput: To de

In [12]:
pred[1]

[{'input': 'Definition: In this task the focus is on physical knowledge about the world. Given the provided goal task in the input, describe a process that would lead to the asked outcome. This process often involves physical motions with objects, such as moving them, arranging them in a certain way, mixing them, shaking them, etc.\n\nPositive Example 1 -\nInput: How do you remove keys from a computer keyboard for cleaning?\nOutput: Place the tip of a knife under the corner of a key and gently twist to pop it off.\n\nPositive Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: by arranging paper towels on the counter and evenly spacing out each key to dry, face side up, on top of them.\n\nNegative Example 1 -\nInput: How do you remove keys?\nOutput: pry them off with a knife.\n\nNegative Example 2 -\nInput: How can you dry wet replacement computer keyboard keys?\nOutput: Place them in the oven on low\n\nNow complete the following example -\nInput: Clean